# SQL Server 2019 Dynamic Data Masking

This notebook contains an example of how to use Dynamic Data Mapping.

<span style="color: var(--vscode-foreground);">This notebook uses the sample AdventureWorks sample database but can be used with any user database.</span>

## Step 0: Create a table, and add data

Execute the following T-SQL to create a table and insert rows in the database. Note the column definition MASKED WITH.

| Function | Description | Examples |
| --- | --- | --- |
| Default | Full masking according to the data types of the designated fields.<br><br>For string data types, use `XXXX` (or fewer) if the size of the field is fewer than 4 characters (**char**, **nchar**, **varchar**, **nvarchar**, **text**, **ntext**).<br><br>For numeric data types use a zero value (**bigint**, **bit**, **decimal**, **int**, **money**, **numeric**, **smallint**, **smallmoney**, **tinyint**, **float**, **real**).<br><br>For date and time data types, use `1900-01-01 00:00:00.0000000` (**date**, **datetime2**, **datetime**, **datetimeoffset**, **smalldatetime**, **time**).<br><br>For binary data types use a single byte of ASCII value 0 (**binary**, **varbinary**, **image**). | Example column definition syntax: `Phone# varchar(12) MASKED WITH (FUNCTION = 'default()') NULL`<br><br>Example of alter syntax: `ALTER COLUMN Gender ADD MASKED WITH (FUNCTION = 'default()')` |
| Email | Masking method that exposes the first letter of an email address and the constant suffix ".com", in the form of an email address. `aXXX@XXXX.com`. | Example definition syntax: `Email varchar(100) MASKED WITH (FUNCTION = 'email()') NULL`<br><br>Example of alter syntax: `ALTER COLUMN Email ADD MASKED WITH (FUNCTION = 'email()')` |
| Random | A random masking function for use on any numeric type to mask the original value with a random value within a specified range. | Example definition syntax: `Account_Number bigint MASKED WITH (FUNCTION = 'random([start range], [end range])')`<br><br>Example of alter syntax: `ALTER COLUMN [Month] ADD MASKED WITH (FUNCTION = 'random(1, 12)')` |
| Custom String | Masking method that exposes the first and last letters and adds a custom padding string in the middle. `prefix,[padding],suffix`<br><br>If the original value is too short to complete the entire mask, part of the prefix or suffix isn't exposed. | Example definition syntax: `FirstName varchar(100) MASKED WITH (FUNCTION = 'partial(prefix,[padding],suffix)') NULL`<br><br>Example of alter syntax: `ALTER COLUMN [Phone Number] ADD MASKED WITH (FUNCTION = 'partial(1,"XXXXXXX",0)')`<br><br>This turns a phone number like `555.123.1234` into `5XXXXXXX`.<br><br>Additional example:<br><br>`ALTER COLUMN [Phone Number] ADD MASKED WITH (FUNCTION = 'partial(5,"XXXXXXX",0)')`<br><br>This turns a phone number like `555.123.1234` into `555.1XXXXXXX`. |
| Datetime | **Applies to:** SQL Server 2022 (16.x)<br><br>Masking method for column defined with data type **datetime**, **datetime2**, **date**, **time**, **datetimeoffset**, **smalldatetime**. It helps masking the `year => datetime("Y")`, `month=> datetime("M")` , `day=>datetime("D")`, `hour=>datetime("h")`, `minute=>datetime("m")`, or `seconds=>datetime("s")` portion of the day. | Example of how to mask the year of the **datetime** value:<br><br>`ALTER COLUMN BirthDay ADD MASKED WITH (FUNCTION = 'datetime("Y")')`<br><br>Example of how to mask the month of the **datetime** value:<br><br>`ALTER COLUMN BirthDay ADD MASKED WITH (FUNCTION = 'datetime("M")')`<br><br>Example of how to mask the minute of the **datetime** value:<br><br>`ALTER COLUMN BirthDay ADD MASKED WITH (FUNCTION = 'datetime("m")')` |

In [ ]:
-- table with masked columns
CREATE TABLE Person.Membership (
    MemberID INT IDENTITY(1, 1) NOT NULL PRIMARY KEY CLUSTERED,
    FirstName VARCHAR(100) MASKED WITH (FUNCTION = 'partial(1, "xxxxx", 1)') NULL,
    LastName VARCHAR(100) NOT NULL,
    Phone VARCHAR(12) MASKED WITH (FUNCTION = 'default()') NULL,
    Email VARCHAR(100) MASKED WITH (FUNCTION = 'email()') NOT NULL,
    DiscountCode SMALLINT MASKED WITH (FUNCTION = 'random(1, 100)') NULL
);


## Step 1: Insert the sample data

In [ ]:

-- inserting sample data
INSERT INTO Person.Membership (FirstName, LastName, Phone, Email, DiscountCode)
VALUES
('Roberto', 'Tamburello', '555.123.4567', 'RTamburello@contoso.com', 10),
('Janice', 'Galvin', '555.123.4568', 'JGalvin@contoso.com.co', 5),
('Shakti', 'Menon', '555.123.4570', 'SMenon@contoso.net', 50),
('Zheng', 'Mu', '555.123.4569', 'ZMu@contoso.net', 40);
GO


## Step 3: Create an unprivileged user.

In [ ]:
-- Create a login for the user
CREATE LOGIN unprivilegeduser
WITH PASSWORD = 'Unpr1v1l3g3d';

-- Create a user in the AdventureWorks database
USE AdventureWorks;
CREATE USER unprivilegeduser
FOR LOGIN unprivilegeduser;

-- Grant necessary permissions to the user
-- Adjust the permissions as needed for your specific requirements
EXEC sp_addrolemember N'db_datareader', N'unprivilegeduser';

## Step 3: Create another Connection to the database as the Unprivileged user

## Step 4: In the new connection, create a new query from AdventureWorks database.

In [ ]:
SELECT [MemberID]
      ,[FirstName]
      ,[LastName]
      ,[Phone]
      ,[Email]
      ,[DiscountCode]
  FROM [AdventureWorks].[Person].[Membership]